# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [9]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [10]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [11]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [12]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [13]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [14]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [15]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [16]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

c:\Users\iliya\Downloads\course-1\venv\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [17]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 17s 18ms/step - loss: 0.6674 - accuracy: 0.6176 - val_loss: 0.6156 - val_accuracy: 0.6732
Epoch 2/10
782/782 [==============================] - 14s 18ms/step - loss: 0.5710 - accuracy: 0.7062 - val_loss: 0.5595 - val_accuracy: 0.7075
Epoch 3/10
782/782 [==============================] - 15s 19ms/step - loss: 0.5146 - accuracy: 0.7478 - val_loss: 0.5274 - val_accuracy: 0.7332
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4758 - accuracy: 0.7750 - val_loss: 0.5071 - val_accuracy: 0.7442
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4464 - accuracy: 0.7936 - val_loss: 0.4883 - val_accuracy: 0.7579
Epoch 6/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4239 - accuracy: 0.8073 - val_loss: 0.4755 - val_accuracy: 0.7664
Epoch 7/10
782/782 [==============================] - 15s 19ms/step - loss: 0.4062 - accuracy: 0.8162 - val_loss: 0.4666 - val_accuracy:

In [18]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 5s 7ms/step - loss: 0.4563 - accuracy: 0.7838
Test score: 0.456304132938385
Test accuracy: 0.783840000629425


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [19]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [20]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [21]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [22]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 31s 37ms/step - loss: 0.6086 - accuracy: 0.6664 - val_loss: 0.5212 - val_accuracy: 0.7472
Epoch 2/10
782/782 [==============================] - 24s 31ms/step - loss: 0.4626 - accuracy: 0.7834 - val_loss: 0.4704 - val_accuracy: 0.7688
Epoch 3/10
782/782 [==============================] - 30s 38ms/step - loss: 0.4016 - accuracy: 0.8222 - val_loss: 0.4531 - val_accuracy: 0.7894
Epoch 4/10
782/782 [==============================] - 41s 52ms/step - loss: 0.3603 - accuracy: 0.8432 - val_loss: 0.4082 - val_accuracy: 0.8122
Epoch 5/10
782/782 [==============================] - 62s 80ms/step - loss: 0.3305 - accuracy: 0.8579 - val_loss: 0.3941 - val_accuracy: 0.8192
Epoch 6/10
782/782 [==============================] - 61s 78ms/step - loss: 0.3076 - accuracy: 0.8700 - val_loss: 0.4007 - val_accuracy: 0.8179
Epoch 7/10
782/782 [==============================] - 52s 67ms/step - loss: 0.2901 - accuracy: 0.8797 - val_loss: 0.3739 - val_accuracy:

In [23]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [24]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [25]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [26]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 36s 43ms/step - loss: 0.6584 - accuracy: 0.6067 - val_loss: 0.6002 - val_accuracy: 0.6816
Epoch 2/10
782/782 [==============================] - 31s 40ms/step - loss: 0.5386 - accuracy: 0.7320 - val_loss: 0.5329 - val_accuracy: 0.7194
Epoch 3/10
782/782 [==============================] - 37s 48ms/step - loss: 0.4805 - accuracy: 0.7718 - val_loss: 0.4887 - val_accuracy: 0.7622
Epoch 4/10
782/782 [==============================] - 51s 66ms/step - loss: 0.4397 - accuracy: 0.7985 - val_loss: 0.4741 - val_accuracy: 0.7818
Epoch 5/10
782/782 [==============================] - 52s 66ms/step - loss: 0.4083 - accuracy: 0.8187 - val_loss: 0.4418 - val_accuracy: 0.7944
Epoch 6/10
782/782 [==============================] - 47s 60ms/step - loss: 0.3883 - accuracy: 0.8279 - val_loss: 0.4371 - val_accuracy: 0.7916
Epoch 7/10
782/782 [==============================] - 28s 36ms/step - loss: 0.3735 - accuracy: 0.8360 - val_loss: 0.4179 - val_accuracy:

In [27]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 35s 45ms/step - loss: 0.3375 - accuracy: 0.8563 - val_loss: 0.4006 - val_accuracy: 0.8162
Epoch 2/10
782/782 [==============================] - 42s 53ms/step - loss: 0.3308 - accuracy: 0.8600 - val_loss: 0.4010 - val_accuracy: 0.8179
Epoch 3/10
782/782 [==============================] - 48s 61ms/step - loss: 0.3246 - accuracy: 0.8635 - val_loss: 0.3911 - val_accuracy: 0.8229
Epoch 4/10
782/782 [==============================] - 42s 54ms/step - loss: 0.3196 - accuracy: 0.8648 - val_loss: 0.3904 - val_accuracy: 0.8238
Epoch 5/10
782/782 [==============================] - 42s 54ms/step - loss: 0.3148 - accuracy: 0.8677 - val_loss: 0.3830 - val_accuracy: 0.8271
Epoch 6/10
782/782 [==============================] - 45s 57ms/step - loss: 0.3097 - accuracy: 0.8699 - val_loss: 0.3808 - val_accuracy: 0.8290
Epoch 7/10
782/782 [==============================] - 43s 55ms/step - loss: 0.3051 - accuracy: 0.8725 - val_loss: 0.3769 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation